
#コモナド

コモナドはモナドの圏論的双対

###コモナドの定義
```
w a -> a (a -> b) ◦ (b -> c)

(=>=) :: (w a -> b) -> (w b -> c) -> (w a -> c)
extract   :: w a -> a
duplicate :: w a -> w (w a)
extend    :: (w b -> a) -> w b -> w a

duplicate = extend id
extend f  = fmap f . duplicate
```

###モナドの定義

```
return :: a -> m a
(>>=)  :: m a -> (a -> m b) -> m b

join   :: m => m (m a) -> m a
join x = x >>= id
```

###モナドとコモナドの双対性

```
             comonad                   |              monad
---------------------------------------|--------------------------------------
extract   :: w a -> a                  | return  :: a -> m a
duplicate :: w a -> w (w a)            | join    :: m (m a) -> m a
(=>>)     :: w a -> (w a -> b) -> w b  | (>>=)   :: m a -> (a -> m b) -> m b
```

##コモナドの実用例
###ストリーム
無限の長さを持つリストと考えられる  
通常のリストは空リストが存在するため、`extract`が実現できないので、`Comonad`インスタンスにならないことに注意が必要
```
data Stream a = Cons a (Stream a)

instance Functor Stream where
  fmap f (Cons x xs) = Cons (f xs) (fmap f xs)

instance Comonad Stream where
  extract (Cons x _) = x
  duplicate xs@(Cons _ xs') = Cons xs (duplicate xs')
  extend f xs@(Cons _ xs') = Cons (f xs) (extend f xs')
```

###ライフゲーム
コモナドはライフゲームに生かされている  
ライフゲームのルールは次の4つ  
・誕生：死んでいるセルに隣接する生きたセルがちょうど3つあれば、次の世代は誕生する  
・生存：生きているセルに隣接するセルが二つか三つならば、次の世代は生存する。  
・過疎：生きているセルに隣接する生きたセルが一つ以下ならば、過疎により死滅する。  
・過密：生きているセルに隣接するセルが四つ以上ならば、過密により死滅する。  

ライフゲームのデータ構造
```
-- | 1点に注目した無限リスト
data Z a = Z [a] a [a]

left, right :: Z a -> Z a
left  (Z (l:ls) c rs    ) = Z ls     l (c:rs)
right (Z ls     c (r:rs)) = Z (c:ls) r rs
```
```
iterate1 :: (a -> a) -> a -> [a]
iterate1 f = tail . iterate f

instance Functor Z where
  fmap f (Z ls c rs) = Z (fmap f ls) (f c) (fmap f rs)

instance Comonad Z where
  -- extract   :: Z a -> a
  extract (Z _ a _) = a

  -- duplicate :: Z a -> Z (Z a)
  duplicate z = Z (iterate1 left z) z (iterate1 right z)
```

```
-- | 1点に注目した無限平面（2重リスト）
newtype Z2 a = Z2 (Z (Z a))

instance Functor Z2 where
  fmap f (Z2 zz) = Z2 (fmap (fmap f) zz)

instance Comonad Z2 where
  -- extract   :: Z2 a -> a
  extract (Z2 zz) = extract (extract zz)

  -- duplicate :: Z2 a -> Z2 (Z2 a)
  duplicate (Z2 zz) = fmap Z2 . Z2 . roll $ roll zz
    where
      roll :: Z (Z a) -> Z (Z (Z a))
      roll zz = Z (iterate1 (fmap left) zz) zz (iterate1 (fmap right) zz)
```
```
-- | 注目している点の周りのTrueの数を数える
neighbours :: Z2 Bool -> Int
neighbours (Z2 (Z
  (Z (n0:_) n1 (n2: _):_)
  (Z (n3:_) _  (n4:_))
  (Z (n5:_) n6 (n7: _):_))) =
    length $ filter id [n0, n1, n2, n3, n4, n5, n6, n7]

-- | 注目している点が次のステップで生存するかを判定する
life :: Z2 Bool -> Bool
life z = (a && (n == 2 || n == 3)) || (not a && n == 3)
  where
    a = extract z
    n = neighbours z
```
```
-- ライフゲームを時間発展させる
extend life :: Z2 Bool -> Z2 Bool
```

ライフゲームの描画
```
wWidth, wHeight :: Num a => a
wWidth  = 640
wHeight = 480

type Model = Z2 Bool

draw :: Model -> Picture
draw (Z2 (Z _ _ rows)) =
  let cSize   = 20 -- セルの大きさ
      -- 基本となるセルの四角形。右下に原点がくる
      cell    = translate (-cSize / 2) (cSize / 2) $ rectangleSolid cSize cSize
      b2c b   = if b then black else white -- Boolから色への変換
      nWidth  = ceiling $ wWidth  / cSize  -- 横方向のセルの個数
      nHeight = ceiling $ wHeight / cSize  -- 縦方向のセルの個数
      cells   = do
        ((Z _ _ row), h) <- zip rows [1..nHeight] -- 縦方向に盤面を走査する
        (b,           w) <- zip row  [1..nWidth]  -- 横方向に行を走査する
        let x = fromIntegral w * cSize - wWidth / 2   -- セルのx座標
            y = wHeight / 2 - fromIntegral h * cSize  -- セルのy座標
            transform = color (b2c b) . translate x y -- セルに施す変形
        pure $ transform cell
   in mconcat cells -- 計算した全てのセルを結合して1つのPictureにする
```
ライフゲームの実装
```
toZ :: a -> [a] -> Z a
toZ a xs = Z (repeat a) a (xs ++ repeat a)

toZ2 :: a -> [[a]] -> Z2 a
toZ2 a xss = Z2 $ toZ (toZ a []) (map (toZ a) xss)


main :: IO ()
main = simulate inWindow white 3 initModel draw (\_ _ -> extend life)
  where
    inWindow  = InWindow "Haskell Day 2018" (wWidth, wHeight) (100, 100)
    field = [ " # "
            , "  #"
            , "###"
            ]
    initModel = toZ2 False $ map (map (== '#')) field
```


